In [ ]:
!jupyter nbconvert --to html "C:/xampp/htdocs/ml_personalNGroup/Group/challenge3/notebooks/02-feature-engineering.ipynb"

## Tải dữ liệu

In [2]:
import pandas as pd

df_train = pd.read_csv('../data/01_raw/train.csv')
df_test = pd.read_csv('../data/01_raw/test.csv')
test_ids = df_test['Id']

In [45]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  3600 non-null   int64  
 1   Artist Name         3600 non-null   object 
 2   Track Name          3600 non-null   object 
 3   Popularity          3505 non-null   float64
 4   danceability        3600 non-null   float64
 5   energy              3600 non-null   float64
 6   key                 3195 non-null   float64
 7   loudness            3600 non-null   float64
 8   mode                3600 non-null   int64  
 9   speechiness         3600 non-null   float64
 10  acousticness        3600 non-null   float64
 11  instrumentalness    2764 non-null   float64
 12  liveness            3600 non-null   float64
 13  valence             3600 non-null   float64
 14  tempo               3600 non-null   float64
 15  duration_in min/ms  3600 non-null   float64
 16  time_s

Xử lý missing values
Sửa dữ liệu lỗi (duration, loudness)
Log transform các feature skew
One-hot encode các cột categorical
Clip outliers hợp lý
Normalize numerical features nếu dùng SVM/KNN
Xử lý imbalance bằng class-weight hoặc SMOTE
Drop hoặc encode Artist Name tùy chiến lược

## Xử lý Missing Values

In [3]:
from datetime import datetime
import openpyxl
from openpyxl import Workbook
import os

# Đường dẫn file log
log_file = '../reports/feature-engineering-log.xlsx'

# Tạo file mới nếu chưa tồn tại
if not os.path.exists(log_file):
    wb = Workbook()
    ws = wb.active
    ws.title = "Process Log"
    ws.append(['Thời gian', 'Hành động', 'Kết luận'])
    wb.save(log_file)

# Mở file và ghi log
wb = openpyxl.load_workbook(log_file)
ws = wb.active

# Thông tin log
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
action = 'Xử lý Missing Values: Điền giá trị thiếu cho Popularity (mean), key (mode), instrumentalness (0)'
conclusion = f'Train: Popularity={df_train["Popularity"].isnull().sum()}, key={df_train["key"].isnull().sum()}, instrumentalness={df_train["instrumentalness"].isnull().sum()} missing | Test: Popularity={df_test["Popularity"].isnull().sum()}, key={df_test["key"].isnull().sum()}, instrumentalness={df_test["instrumentalness"].isnull().sum()} missing'

# Thêm dòng mới
ws.append([current_time, action, conclusion])

# Lưu file
wb.save(log_file)
print(f"Đã ghi log vào {log_file}")

# Thực hiện xử lý missing values
df_train['Popularity'] = df_train['Popularity'].fillna(df_train['Popularity'].mean())
df_train['key'] = df_train['key'].fillna(df_train['key'].mode()[0])
df_train['instrumentalness'] = df_train['instrumentalness'].fillna(0)
df_test['key'] = df_test['key'].fillna(df_test['key'].mode()[0])
df_test['instrumentalness'] = df_test['instrumentalness'].fillna(0)
df_test['Popularity'] = df_test['Popularity'].fillna(df_train['Popularity'].mean())

Đã ghi log vào ../reports/feature-engineering-log.xlsx


In [47]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14396 entries, 0 to 14395
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  14396 non-null  int64  
 1   Artist Name         14396 non-null  object 
 2   Track Name          14396 non-null  object 
 3   Popularity          14396 non-null  float64
 4   danceability        14396 non-null  float64
 5   energy              14396 non-null  float64
 6   key                 14396 non-null  float64
 7   loudness            14396 non-null  float64
 8   mode                14396 non-null  int64  
 9   speechiness         14396 non-null  float64
 10  acousticness        14396 non-null  float64
 11  instrumentalness    14396 non-null  float64
 12  liveness            14396 non-null  float64
 13  valence             14396 non-null  float64
 14  tempo               14396 non-null  float64
 15  duration_in min/ms  14396 non-null  float64
 16  time

## Sửa lỗi dữ liệu

(a) Cột duration_in min/ms bị lẫn đơn vị

In [4]:
# Mở file và ghi log
wb = openpyxl.load_workbook(log_file)
ws = wb.active

# Thông tin log
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
action = 'Kiểm tra dữ liệu duration_in min/ms trước khi chuẩn hóa đơn vị'
conclusion = f'Train: min={df_train["duration_in min/ms"].min():.2f}, max={df_train["duration_in min/ms"].max():.2f}, mean={df_train["duration_in min/ms"].mean():.2f} | Test: min={df_test["duration_in min/ms"].min():.2f}, max={df_test["duration_in min/ms"].max():.2f}, mean={df_test["duration_in min/ms"].mean():.2f}'

# Thêm dòng mới
ws.append([current_time, action, conclusion])

# Lưu file
wb.save(log_file)
print(f"Đã ghi log vào {log_file}")

# Hiển thị dữ liệu
print("\nTrain duration_in min/ms:")
print(df_train['duration_in min/ms'].describe())
print("\nTest duration_in min/ms:")
print(df_test['duration_in min/ms'].describe())

Đã ghi log vào ../reports/feature-engineering-log.xlsx

Train duration_in min/ms:
count    1.439600e+04
mean     2.000942e+05
std      1.116891e+05
min      5.016500e-01
25%      1.654458e+05
50%      2.089410e+05
75%      2.522470e+05
max      1.477187e+06
Name: duration_in min/ms, dtype: float64

Test duration_in min/ms:
count    3.600000e+03
mean     2.033447e+05
std      1.131593e+05
min      9.871167e-01
25%      1.688360e+05
50%      2.103355e+05
75%      2.533958e+05
max      1.412451e+06
Name: duration_in min/ms, dtype: float64


In [ ]:
# Mở file và ghi log
wb = openpyxl.load_workbook(log_file)
ws = wb.active

# Thông tin log
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
action = 'Chuẩn hóa đơn vị duration_in min/ms: Chuyển giá trị < 30 (phút) sang milliseconds, đổi tên cột thành duration_in_ms'
conclusion = f'Train: Đã chuyển {mask_min.sum()} giá trị từ phút sang ms | Test: Đã chuyển {mask_min_test.sum()} giá trị từ phút sang ms'

# Thêm dòng mới
ws.append([current_time, action, conclusion])

# Lưu file
wb.save(log_file)
print(f"Đã ghi log vào {log_file}")

# Thực hiện chuẩn hóa đơn vị
mask_min = df_train['duration_in min/ms'] < 30
df_train.loc[mask_min, 'duration_in min/ms'] *= 60*1000
df_train.rename(columns={'duration_in min/ms': 'duration_in_ms'}, inplace=True)

mask_min_test = df_test['duration_in min/ms'] < 30
df_test.loc[mask_min_test, 'duration_in min/ms'] *= 60*1000
df_test.rename(columns={'duration_in min/ms': 'duration_in_ms'}, inplace=True)

print(f"\nTrain: Đã chuyển {mask_min.sum()} giá trị")
print(f"Test: Đã chuyển {mask_min_test.sum()} giá trị")

Đã ghi log vào ../reports/feature-engineering-log.xlsx

Train: Đã chuyển 2075 giá trị
Test: Đã chuyển 505 giá trị


(b) Cột loudness có giá trị dương

In [7]:
# Mở file và ghi log
wb = openpyxl.load_workbook(log_file)
ws = wb.active

# Thông tin log
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
action = 'Sửa lỗi dữ liệu loudness: Clip các giá trị dương về 0 (loudness phải âm hoặc 0)'
positive_train = (df_train['loudness'] > 0).sum()
positive_test = (df_test['loudness'] > 0).sum()

# Thực hiện clip
df_train['loudness'] = df_train['loudness'].clip(upper=0)
df_test['loudness'] = df_test['loudness'].clip(upper=0)

conclusion = f'Train: Đã clip {positive_train} giá trị dương về 0 | Test: Đã clip {positive_test} giá trị dương về 0'

# Thêm dòng mới
ws.append([current_time, action, conclusion])

# Lưu file
wb.save(log_file)
print(f"Đã ghi log vào {log_file}")
print(f"\nTrain: Đã clip {positive_train} giá trị dương")
print(f"Test: Đã clip {positive_test} giá trị dương")

Đã ghi log vào ../reports/feature-engineering-log.xlsx

Train: Đã clip 7 giá trị dương
Test: Đã clip 1 giá trị dương


Kiểm tra còn gtri nào lớn hơn 0 kh 

In [8]:
# Mở file và ghi log
wb = openpyxl.load_workbook(log_file)
ws = wb.active

# Thông tin log
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
action = 'Kiểm tra loudness sau khi clip: Xác nhận không còn giá trị dương'
train_positive = len(df_train[df_train['loudness'] > 0])
test_positive = len(df_test[df_test['loudness'] > 0])
conclusion = f'Train: {train_positive} giá trị dương còn lại | Test: {test_positive} giá trị dương còn lại'

# Thêm dòng mới
ws.append([current_time, action, conclusion])

# Lưu file
wb.save(log_file)
print(f"Đã ghi log vào {log_file}")
print(f"\nTrain: {train_positive} giá trị loudness > 0")
print(f"Test: {test_positive} giá trị loudness > 0")

Đã ghi log vào ../reports/feature-engineering-log.xlsx

Train: 0 giá trị loudness > 0
Test: 0 giá trị loudness > 0


## Biến đổi phân phối lệch

In [11]:
import numpy as np

# Định nghĩa các cột cần transform
skew_cols = ['speechiness', 'acousticness', 'liveness', 'instrumentalness']

# Mở file và ghi log
wb = openpyxl.load_workbook(log_file)
ws = wb.active

# Thông tin log
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
action = 'Log transform các feature skew: Áp dụng log1p cho speechiness, acousticness, liveness, instrumentalness'
conclusion = f'Đã transform {len(skew_cols)} cột có phân phối lệch: {", ".join(skew_cols)}'

# Thêm dòng mới
ws.append([current_time, action, conclusion])

# Lưu file
wb.save(log_file)
print(f"Đã ghi log vào {log_file}")

# Thực hiện log transform
for col in skew_cols:
    df_train[col] = np.log1p(df_train[col])
    df_test[col] = np.log1p(df_test[col])

print(f"\nĐã áp dụng log1p cho {len(skew_cols)} cột: {', '.join(skew_cols)}")

Đã ghi log vào ../reports/feature-engineering-log.xlsx

Đã áp dụng log1p cho 4 cột: speechiness, acousticness, liveness, instrumentalness


## Encoding biến phân loại

In [12]:
# Mở file và ghi log
wb = openpyxl.load_workbook(log_file)
ws = wb.active

# Thông tin log
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
action = 'Encoding biến phân loại: One-hot encode key, mode, time_signature (drop_first=True) và xóa cột Track Name'
conclusion = f'Train: {len(df_train.columns)} cột sau encoding | Test: {len(df_test.columns)} cột sau encoding. Đã xóa Track Name khỏi cả 2 datasets'

# Thêm dòng mới
ws.append([current_time, action, conclusion])

# Lưu file
wb.save(log_file)
print(f"Đã ghi log vào {log_file}")

# Thực hiện encoding
df_train = pd.get_dummies(df_train, columns=['key', 'mode', 'time_signature'], drop_first=True)
df_train.drop(columns=['Track Name'], inplace=True) 
df_test = pd.get_dummies(df_test, columns=['key', 'mode', 'time_signature'], drop_first=True)
df_test.drop(columns=['Track Name'], inplace=True)

print(f"\nTrain shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

Đã ghi log vào ../reports/feature-engineering-log.xlsx

Train shape: (14396, 28)
Test shape: (3600, 27)


In [54]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14396 entries, 0 to 14395
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                14396 non-null  int64  
 1   Artist Name       14396 non-null  object 
 2   Popularity        14396 non-null  float64
 3   danceability      14396 non-null  float64
 4   energy            14396 non-null  float64
 5   loudness          14396 non-null  float64
 6   speechiness       14396 non-null  float64
 7   acousticness      14396 non-null  float64
 8   instrumentalness  14396 non-null  float64
 9   liveness          14396 non-null  float64
 10  valence           14396 non-null  float64
 11  tempo             14396 non-null  float64
 12  duration_in_ms    14396 non-null  float64
 13  Class             14396 non-null  int64  
 14  key_2.0           14396 non-null  bool   
 15  key_3.0           14396 non-null  bool   
 16  key_4.0           14396 non-null  bool  

## Xử lý Outliers

Instrumentalness outlier theo IQR = "outlier giả" → KHÔNG XOÁ

tempo có outlier thật nhưng ít → có thể clip

duration_ms sau chuẩn hóa nên clip 1%–99%

In [13]:
# Mở file và ghi log
wb = openpyxl.load_workbook(log_file)
ws = wb.active

# Thông tin log
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
action = 'Xử lý Outliers: Clip tempo và duration_in_ms về quantile 1%-99% để loại bỏ outliers cực đoan'

# Đếm số lượng outliers trước khi clip
train_tempo_outliers = ((df_train['tempo'] < df_train['tempo'].quantile(0.01)) | 
                        (df_train['tempo'] > df_train['tempo'].quantile(0.99))).sum()
train_duration_outliers = ((df_train['duration_in_ms'] < df_train['duration_in_ms'].quantile(0.01)) | 
                           (df_train['duration_in_ms'] > df_train['duration_in_ms'].quantile(0.99))).sum()
test_tempo_outliers = ((df_test['tempo'] < df_test['tempo'].quantile(0.01)) | 
                       (df_test['tempo'] > df_test['tempo'].quantile(0.99))).sum()
test_duration_outliers = ((df_test['duration_in_ms'] < df_test['duration_in_ms'].quantile(0.01)) | 
                          (df_test['duration_in_ms'] > df_test['duration_in_ms'].quantile(0.99))).sum()

# Thực hiện clip
df_train['tempo'] = df_train['tempo'].clip(lower=df_train['tempo'].quantile(0.01),
                                            upper=df_train['tempo'].quantile(0.99))
df_train['duration_in_ms'] = df_train['duration_in_ms'].clip(lower=df_train['duration_in_ms'].quantile(0.01),
                                                              upper=df_train['duration_in_ms'].quantile(0.99))
df_test['tempo'] = df_test['tempo'].clip(lower=df_test['tempo'].quantile(0.01),
                                          upper=df_test['tempo'].quantile(0.99))
df_test['duration_in_ms'] = df_test['duration_in_ms'].clip(lower=df_test['duration_in_ms'].quantile(0.01),
                                                            upper=df_test['duration_in_ms'].quantile(0.99))

conclusion = f'Train: Đã clip {train_tempo_outliers} outliers tempo, {train_duration_outliers} outliers duration_in_ms | Test: Đã clip {test_tempo_outliers} outliers tempo, {test_duration_outliers} outliers duration_in_ms'

# Thêm dòng mới
ws.append([current_time, action, conclusion])

# Lưu file
wb.save(log_file)
print(f"Đã ghi log vào {log_file}")
print(f"\nTrain: Đã xử lý {train_tempo_outliers} outliers tempo và {train_duration_outliers} outliers duration_in_ms")
print(f"Test: Đã xử lý {test_tempo_outliers} outliers tempo và {test_duration_outliers} outliers duration_in_ms")

Đã ghi log vào ../reports/feature-engineering-log.xlsx

Train: Đã xử lý 288 outliers tempo và 288 outliers duration_in_ms
Test: Đã xử lý 72 outliers tempo và 72 outliers duration_in_ms


In [56]:
df_train[['tempo', 'duration_in_ms']].describe()

,tempo,duration_in_ms
count,14396.000000,14396.000000
mean,122.690984,233627.485135
std,29.192225,72059.831562
min,70.915650,113670.250000
25%,99.799000,187333.000000
50%,120.060000,219800.500000
75%,141.988250,262455.000000
max,193.754100,523353.000000


## Feature Scaling (Chuẩn hóa dữ liệu)

In [14]:
# Mở file và ghi log
wb = openpyxl.load_workbook(log_file)
ws = wb.active

# Thông tin log
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
action = 'Feature Scaling: Chuẩn hóa các feature numerical bằng StandardScaler'

# Thực hiện scaling
from sklearn.preprocessing import StandardScaler

scale_cols = ['danceability','energy','loudness','speechiness','acousticness',
              'instrumentalness','liveness','valence','tempo','duration_in_ms','Popularity']

scaler = StandardScaler()
df_train[scale_cols] = scaler.fit_transform(df_train[scale_cols])
df_test[scale_cols] = scaler.transform(df_test[scale_cols])

conclusion = f'Đã chuẩn hóa {len(scale_cols)} cột numerical: {", ".join(scale_cols[:5])}... với mean=0, std=1'

# Thêm dòng mới
ws.append([current_time, action, conclusion])

# Lưu file
wb.save(log_file)
print(f"Đã ghi log vào {log_file}")
print(f"\nĐã chuẩn hóa {len(scale_cols)} cột")

Đã ghi log vào ../reports/feature-engineering-log.xlsx

Đã chuẩn hóa 11 cột


In [58]:
df_train.head()

,Id,Artist Name,Popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,...,key_6.0,key_7.0,key_8.0,key_9.0,key_10.0,key_11.0,mode_1,time_signature_3,time_signature_4,time_signature_5
0,1,Marina Maximilian,-0.437113,-1.263390,-0.535777,0.308703,-0.519565,0.562430,-0.508555,-0.590149,...,False,False,False,True,False,False,False,False,True,False
1,2,The Black Keys,1.305483,1.098986,0.358445,0.580863,0.134101,-0.733841,-0.282518,-0.605319,...,False,False,False,False,False,True,True,False,True,False
2,3,Royal & the Serpent,0.000000,0.247081,0.600012,0.445522,-0.201475,-0.847979,1.921148,0.660580,...,False,True,False,False,False,False,True,False,True,False
3,4,Detroit Blues Band,-1.889277,-0.169809,-1.502046,-1.678758,-0.613149,1.995926,-0.404395,0.764374,...,False,True,False,False,False,False,True,True,False,False
4,5,Coast Contra,0.201839,0.132285,0.485585,0.691551,2.075815,-0.110772,-0.508555,0.169997,...,True,False,False,False,False,False,False,False,True,False


## Xử lý Imbalanced Class (Class 10 quá nhiều)

In [16]:
from sklearn.utils.class_weight import compute_class_weight

# Mở file và ghi log
wb = openpyxl.load_workbook(log_file)
ws = wb.active

# Thông tin log
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
action = 'Xử lý Imbalanced Class: Tính toán class weights để xử lý mất cân bằng dữ liệu (Class 10 quá nhiều)'

# Tính toán class weights
classes = np.unique(df_train['Class'])
weights = compute_class_weight(class_weight='balanced', classes=classes, y=df_train['Class'])
class_weights = dict(zip(classes, weights))

# Đếm số lượng mỗi class
class_counts = df_train['Class'].value_counts().sort_index()
conclusion = f'Đã tính class_weights cho {len(classes)} classes. Class phổ biến nhất: Class {class_counts.idxmax()} ({class_counts.max()} samples). Weights sẽ được dùng trong training'

# Thêm dòng mới
ws.append([current_time, action, conclusion])

# Lưu file
wb.save(log_file)
print(f"Đã ghi log vào {log_file}")
print(f"\nClass weights: {class_weights}")
print(f"\nPhân bố classes:\n{class_counts}")

Đã ghi log vào ../reports/feature-engineering-log.xlsx

Class weights: {0: 2.6174545454545455, 1: 1.1919191919191918, 2: 1.2855867119128417, 3: 4.064370412196499, 4: 4.221700879765396, 5: 1.1311385243969514, 6: 0.6325409728019684, 7: 2.838887793334648, 8: 0.8824863605713235, 9: 0.6482056823810167, 10: 0.3305701623458633}

Phân bố classes:
Class
0      500
1     1098
2     1018
3      322
4      310
5     1157
6     2069
7      461
8     1483
9     2019
10    3959
Name: count, dtype: int64


Lưu file csv đã xử lí

In [60]:
import os
os.makedirs("../data/02_processed", exist_ok=True)
df_train.to_csv("../data/02_processed/train_clean.csv", index=False)
df_test.to_csv("../data/02_processed/test_clean.csv", index=False)
print("Dữ liệu đã sẵn sàng cho mô hình (03_processed).")

Dữ liệu đã sẵn sàng cho mô hình (03_processed).
